In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from pprint import pprint
import time

In [2]:
import re
from datetime import datetime

def process_game_price_data(data):
    processed_data = data.copy()

    # '원가'와 '할인가'를 정수형으로 변환
    for key in ['원가', '할인가']:
        if key in processed_data and isinstance(processed_data[key], str):
            # '₩'와 ',' 제거
            cleaned_value = re.sub(r'[₩,]', '', processed_data[key])
            try:
                processed_data[key] = int(cleaned_value)
            except ValueError:
                print(f"경고: '{key}' 값을 정수로 변환할 수 없습니다: {processed_data[key]}")
                processed_data[key] = None

    # '이전과 비교한 할인가격'을 부호가 붙은 정수형으로 변환
    if '이전과 비교한 할인가격' in processed_data and isinstance(processed_data['이전과 비교한 할인가격'], str):
        original_value = processed_data['이전과 비교한 할인가격']
        sign = ''
        cleaned_value_for_sign = original_value # 부호 추출을 위한 임시 변수

        if cleaned_value_for_sign.startswith('+'):
            sign = '+'
            cleaned_value_for_sign = cleaned_value_for_sign[1:] # 부호 제외
        elif cleaned_value_for_sign.startswith('-'):
            sign = '-'
            cleaned_value_for_sign = cleaned_value_for_sign[1:] # 부호 제외
        
        # '₩'와 ',' 제거
        numeric_part = re.sub(r'[₩,]', '', cleaned_value_for_sign)
        try:
            # 부호와 숫자 부분을 합쳐 정수로 변환
            processed_data['이전과 비교한 할인가격'] = int(sign + numeric_part)
        except ValueError:
            print(f"경고: '이전과 비교한 할인가격' 값을 정수로 변환할 수 없습니다: {processed_data['이전과 비교한 할인가격']}")
            processed_data['이전과 비교한 할인가격'] = None

    # '할인 시작 날짜'를 'YYYY-MM-DD' 형식으로 변환
    if '할인 시작 날짜' in processed_data and isinstance(processed_data['할인 시작 날짜'], str):
        try:
            # 원본 날짜 형식에 맞게 파싱
            date_obj = datetime.strptime(processed_data['할인 시작 날짜'], '%d %b %Y, %H:%M')
            processed_data['할인 시작 날짜'] = date_obj.strftime('%Y-%m-%d')
        except ValueError:
            print(f"경고: '할인 시작 날짜' 형식을 변환할 수 없습니다: {processed_data['할인 시작 날짜']}")
            processed_data['할인 시작 날짜'] = None
    
    return processed_data

In [3]:
def history_log(game_name):
    history_url = f"https://isthereanydeal.com/game/{game_name}/history/"
    #print(history_url)

    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install())) #Chrome을 위한 webdriver install
    driver.get(history_url)

    
    time.sleep(5)

    #div 엘리먼트 선택해서 가져오기
    game_log=list()
    div_tags = driver.find_elements(By.CSS_SELECTOR,'div.entry.svelte-17mbxpf')
    # print(len(div_tags))

    for div_tag in div_tags :
        time_tag = div_tag.find_elements(By.CSS_SELECTOR,'span.time.svelte-17mbxpf')
        discount_date = time_tag[0].text

        store_tag = div_tag.find_elements(By.CSS_SELECTOR,'span.svelte-zbwnbn')
        store = store_tag[0].text

        duration_tag = div_tag.find_elements(By.CSS_SELECTOR,'div.duration.svelte-17mbxpf')
        duration = duration_tag[0].text

        cut_tag = div_tag.find_elements(By.CSS_SELECTOR,'span.cut.svelte-17mbxpf')
        cut = cut_tag[1].text

        price_tag = div_tag.find_elements(By.CSS_SELECTOR,'div.price.svelte-17mbxpf div')
        reg_price = price_tag[0].text
        discount_price = price_tag[2].text
        price_comp = price_tag[3].text

        #log data를 이용한 dict만들기
        log = {'할인 시작 날짜':discount_date, '플랫폼 이름': store,'할인 지속시간':duration,'할인율':cut,'원가':reg_price,'할인가':discount_price,'이전과 비교한 할인가격':price_comp}
        processed_log=process_game_price_data(log)
        # pprint(processed_log)

        game_log.append(processed_log)

    #5초로 waiting time 설정
    time.sleep(5)
    #driver 종료
    driver.quit()
    
    return game_log
    
    

In [ ]:
import os
import json

def save_json(data, game_name):
    """JSON 파일 저장 함수"""
    os.makedirs('data_json', exist_ok=True)
    filepath = f'data_json/{game_name}.json'
    with open(filepath, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)

In [ ]:
game_name = 'the-evil-within-2'
game_log = history_log(game_name)
print(game_log)
save_json(game_log,game_name)

In [ ]:
import pandas as pd
def jsonfile_to_csv(game_name):
    gamelog_df = pd.read_json(f'data_json/{game_name}.json')
    game_log.fillna('NoData',inplace = True)
    os.makedirs('data/data_csv', exist_ok=True)
    gamelog_df.to_csv(f'data_csv/{game_name}.csv', index=False)
    return

## 리팩토링 코드

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.common.exceptions import WebDriverException, NoSuchElementException
from pprint import pprint
import time
import re
from datetime import datetime
import os
import json
import pandas as pd

# (이전 코드와 동일하므로 생략)
def process_game_price_data(data):
    processed_data = data.copy()

    # '원가'와 '할인가'를 정수형으로 변환
    for key in ['원가', '할인가']:
        if key in processed_data and isinstance(processed_data[key], str):
            # '₩'와 ',' 제거
            cleaned_value = re.sub(r'[₩,]', '', processed_data[key])
            try:
                processed_data[key] = int(cleaned_value)
            except ValueError:
                print(f"경고: '{key}' 값을 정수로 변환할 수 없습니다: {processed_data[key]}")
                processed_data[key] = None

    # '이전과 비교한 할인가격'을 부호가 붙은 정수형으로 변환
    if '이전과 비교한 할인가격' in processed_data and isinstance(processed_data['이전과 비교한 할인가격'], str):
        original_value = processed_data['이전과 비교한 할인가격']
        sign = ''
        cleaned_value_for_sign = original_value # 부호 추출을 위한 임시 변수

        if cleaned_value_for_sign.startswith('+'):
            sign = '+'
            cleaned_value_for_sign = cleaned_value_for_sign[1:] # 부호 제외
        elif cleaned_value_for_sign.startswith('-'):
            sign = '-'
            cleaned_value_for_sign = cleaned_value_for_sign[1:] # 부호 제외
        
        # '₩'와 ',' 제거
        numeric_part = re.sub(r'[₩,]', '', cleaned_value_for_sign)
        try:
            # 부호와 숫자 부분을 합쳐 정수로 변환
            processed_data['이전과 비교한 할인가격'] = int(sign + numeric_part)
        except ValueError:
            print(f"경고: '이전과 비교한 할인가격' 값을 정수로 변환할 수 없습니다: {processed_data['이전과 비교한 할인가격']}")
            processed_data['이전과 비교한 할인가격'] = None

    # '할인 시작 날짜'를 'YYYY-MM-DD' 형식으로 변환
    if '할인 시작 날짜' in processed_data and isinstance(processed_data['할인 시작 날짜'], str):
        try:
            # 원본 날짜 형식에 맞게 파싱
            date_obj = datetime.strptime(processed_data['할인 시작 날짜'], '%d %b %Y, %H:%M')
            processed_data['할인 시작 날짜'] = date_obj.strftime('%Y-%m-%d')
        except ValueError:
            print(f"경고: '할인 시작 날짜' 형식을 변환할 수 없습니다: {processed_data['할인 시작 날짜']}")
            processed_data['할인 시작 날짜'] = None
    
    return processed_data

def history_log(game_name):
    # 1. game_name 유효성 검사
    if not game_name or not isinstance(game_name, str):
        print("경고: 유효하지 않은 game_name이 입력되었습니다. 'nodata'를 반환합니다.")
        return 'nodata'

    history_url = f"https://isthereanydeal.com/game/{game_name}/history/"
    # print(history_url)

    driver = None # 드라이버 초기화
    try:
        driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
        driver.get(history_url)

        time.sleep(5) # 페이지 로드를 위한 대기

        # 2. 'nodata' 상황 판단: 특정 요소가 존재하지 않는지 확인
        # 'isthereanydeal.com'에서 게임이 없거나 페이지를 찾을 수 없을 때 나타나는 요소나 패턴을 확인합니다.
        # 예를 들어, "Page Not Found" 텍스트나, 데이터가 없는 경우에만 나타나는 특정 div를 찾습니다.
        # 여기서는 history-placeholder 클래스를 가진 div가 있는지 확인합니다.
        try:
            # 게임이 없을 경우 'There are currently no deals for this game.' 메시지를 포함하는
            # div.history-placeholder 요소가 나타납니다.
            placeholder_div = driver.find_element(By.CSS_SELECTOR, 'div.history-placeholder')
            if "no deals for this game" in placeholder_div.text:
                print(f"'{game_name}'에 대한 거래 기록을 찾을 수 없습니다. 'nodata'를 반환합니다.")
                return 'nodata'
        except NoSuchElementException:
            # placeholder_div가 없으면, 게임 데이터가 있을 수 있다는 의미입니다. 계속 진행합니다.
            pass
        
        # 실제 게임 로그 데이터를 담고 있는 div 요소들을 가져옵니다.
        game_log = []
        div_tags = driver.find_elements(By.CSS_SELECTOR, 'div.entry.svelte-17mbxpf')

        if not div_tags: # div_tags가 비어있으면, 게임 데이터가 없는 것으로 판단합니다.
            print(f"'{game_name}'에 대한 거래 기록 div를 찾을 수 없습니다. 'nodata'를 반환합니다.")
            return 'nodata'

        for div_tag in div_tags:
            try:
                time_tag = div_tag.find_elements(By.CSS_SELECTOR, 'span.time.svelte-17mbxpf')
                discount_date = time_tag[0].text if time_tag else None

                store_tag = div_tag.find_elements(By.CSS_SELECTOR, 'span.svelte-zbwnbn')
                store = store_tag[0].text if store_tag else None

                duration_tag = div_tag.find_elements(By.CSS_SELECTOR, 'div.duration.svelte-17mbxpf')
                duration = duration_tag[0].text if duration_tag else None

                cut_tag = div_tag.find_elements(By.CSS_SELECTOR, 'span.cut.svelte-17mbxpf')
                # cut_tag[0]은 할인의 종류 (e.g., "Discount"), cut_tag[1]이 할인율을 나타냅니다.
                cut = cut_tag[1].text if len(cut_tag) > 1 else None

                price_tag = div_tag.find_elements(By.CSS_SELECTOR, 'div.price.svelte-17mbxpf div')
                reg_price = price_tag[0].text if len(price_tag) > 0 else None
                discount_price = price_tag[2].text if len(price_tag) > 2 else None
                price_comp = price_tag[3].text if len(price_tag) > 3 else None

                log = {
                    '할인 시작 날짜': discount_date,
                    '플랫폼 이름': store,
                    '할인 지속시간': duration,
                    '할인율': cut,
                    '원가': reg_price,
                    '할인가': discount_price,
                    '이전과 비교한 할인가격': price_comp
                }
                processed_log = process_game_price_data(log)
                game_log.append(processed_log)
            except Exception as e:
                print(f"개별 로그 처리 중 오류 발생: {e}")
                continue # 오류가 발생해도 다음 로그로 넘어갑니다.

        return game_log

    except WebDriverException as e:
        print(f"웹 드라이버 오류 발생: {e}. 'nodata'를 반환합니다.")
        return 'nodata'
    except Exception as e:
        print(f"예기치 않은 오류 발생: {e}. 'nodata'를 반환합니다.")
        return 'nodata'
    finally:
        if driver:
            driver.quit() # 오류 발생 여부와 상관없이 드라이버 종료

# data디렉토리를 생성하고 data_json 디렉토리에 파일을 넣어주는 함수
def save_json(data, game_name):
    """JSON 파일 저장 함수"""
    if data == 'nodata': # 'nodata'가 반환되면 파일 저장하지 않음
        print(f"'{game_name}'에 대한 데이터가 없어 JSON 파일을 저장하지 않습니다.")
        return

    os.makedirs('data/data_json', exist_ok=True)
    filepath = f'data/data_json/{game_name}.json'
    with open(filepath, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)
    print(f"'{game_name}.json' 파일이 성공적으로 저장되었습니다.")

def jsonfile_to_csv(game_name):
    json_filepath = f'data/data_json/{game_name}.json'
    if not os.path.exists(json_filepath): # JSON 파일이 없으면 CSV 변환하지 않음
        print(f"'{game_name}.json' 파일이 존재하지 않아 CSV로 변환할 수 없습니다.")
        return

    gamelog_df = pd.read_json(json_filepath)
    gamelog_df.fillna('NoData',inplace = True)
    os.makedirs('data/data_csv', exist_ok=True)
    gamelog_df.to_csv(f'data/data_csv/{game_name}.csv', index=False)
    print(f"'{game_name}.csv' 파일이 성공적으로 생성되었습니다.")
    return

def clean_game_name_final(name):
    """
    게임 이름 문자열을 최종 클리닝하는 함수:
    - 모두 소문자로 변환
    - '&'를 'and'로 변환
    - 띄어쓰기를 하이픈으로 변환
    - 영어 소문자, 숫자, 하이픈 외 모든 문자 제거
    - 연속된 하이픈 합치기 및 불필요한 하이픈 제거
    """
    # 0. 입력값이 NaN일 경우 빈 문자열로 처리
    if pd.isna(name):
        return ""
        
    # 문자열로 변환하고 모두 소문자로 변경
    cleaned_name = str(name).lower()

    # 1. '&' 기호를 'and'로 변환 (먼저 처리)
    cleaned_name = cleaned_name.replace('&', 'and')

    # 2. 띄어쓰기(' ')를 하이픈('-')으로 변환
    # 이 단계를 모든 비-a-z0-9- 문자 제거 전에 수행하여 공백이 하이픈으로 변환되도록 합니다.
    cleaned_name = cleaned_name.replace(' ', '-')

    # 3. 영어 소문자, 숫자, 하이픈('-')을 제외한 모든 문자 제거
    # 이제 공백은 하이픈으로 변환되었으므로, 이 정규식은 하이픈을 제외한 나머지 불필요한 문자만 제거합니다.
    cleaned_name = re.sub(r'[^a-z0-9-]', '', cleaned_name)

    # 4. 연속으로 나타나는 하이픈을 하나로 줄이기 (예: 'metal--gear' -> 'metal-gear')
    # 이전 단계에서 공백이 하이픈으로 바뀌고, 다른 문자들이 제거되면서 연속 하이픈이 생길 수 있습니다.
    cleaned_name = re.sub(r'-+', '-', cleaned_name)
    
    # 5. 문장 시작/끝에 불필요하게 붙는 하이픈 제거 (예: '-metal-gear-' -> 'metal-gear')
    cleaned_name = cleaned_name.strip('-')
    
    # 6. 마지막으로, 혹시 남아있을 수 있는 공백 제거 (trim)
    cleaned_name = cleaned_name.strip()

    return cleaned_name

# if __name__ == "__main__":
#     # 케이스 1: 정상적인 게임 이름
#     print("--- Case 1: Normal game name ---")
#     game_name_1 = 'sid-meiers-civilization-vi'
#     gamelog_1 = history_log(game_name_1)
#     if gamelog_1 != 'nodata':
#         save_json(gamelog_1, game_name_1)
#         jsonfile_to_csv(game_name_1)
#     print("\n")

#     # 케이스 2: 존재하지 않는 게임 이름
#     print("--- Case 2: Non-existent game name ---")
#     game_name_2 = 'this-game-does-not-exist-12345'
#     gamelog_2 = history_log(game_name_2)
#     if gamelog_2 != 'nodata':
#         save_json(gamelog_2, game_name_2)
#         jsonfile_to_csv(game_name_2)
#     else:
#         print(f"'{game_name_2}'에 대한 데이터를 찾을 수 없어 'nodata'를 반환했습니다.")
#     print("\n")

#     # 케이스 3: game_name이 빈 문자열인 경우
#     print("--- Case 3: Empty game_name ---")
#     game_name_3 = ''
#     gamelog_3 = history_log(game_name_3)
#     if gamelog_3 != 'nodata':
#         save_json(gamelog_3, game_name_3)
#         jsonfile_to_csv(game_name_3)
#     else:
#         print(f"'{game_name_3}'에 대한 데이터를 찾을 수 없어 'nodata'를 반환했습니다.")
#     print("\n")

#     # 케이스 4: game_name이 None인 경우
#     print("--- Case 4: None game_name ---")
#     game_name_4 = None
#     gamelog_4 = history_log(game_name_4)
#     if gamelog_4 != 'nodata':
#         save_json(gamelog_4, game_name_4)
#         jsonfile_to_csv(game_name_4)
#     else:
#         print(f"'{game_name_4}'에 대한 데이터를 찾을 수 없어 'nodata'를 반환했습니다.")
#     print("\n")

In [ ]:
# 게임 csv파일을 받아서 '게임 이름'을 list로 받기
csvfilename = '../steam_normalized_data.csv'
game_data = pd.read_csv(csvfilename)

game_data['게임 이름'] = game_data['게임 이름'].apply(clean_game_name_final)
games = game_data['게임 이름'].tolist()

#gamelog를 json 및 csv 파일로 저장
for game_name in games[:3] :
    gamelog = history_log(game_name)
    if gamelog != 'nodata':
        save_json(gamelog, game_name)
        jsonfile_to_csv(game_name)
    else:
        print(f"'{game_name}'에 대한 데이터를 찾을 수 없어 'nodata'를 반환했습니다.")

### 안 될 경우 

In [ ]:

# driver = webdriver.Chrome(service=Service(ChromeDriverManager().install())) #Chrome을 위한 webdriver install
# driver.get("https://isthereanydeal.com/game/call-of-duty-modern-warfare-ii/history/")

# time.sleep(5)

# #div 엘리먼트 선택해서 가져오기
# div_tags = driver.find_elements(By.CSS_SELECTOR,'div.entry.svelte-17mbxpf')
# print(len(div_tags))

# for div_tag in div_tags :
#     time_tag = div_tag.find_elements(By.CSS_SELECTOR,'span.time.svelte-17mbxpf')
#     discount_date = time_tag[0].text
#     print(discount_date)

#     store_tag = div_tag.find_elements(By.CSS_SELECTOR,'span.svelte-zbwnbn')
#     store = store_tag[0].text

#     duration_tag = div_tag.find_elements(By.CSS_SELECTOR,'div.duration.svelte-17mbxpf')
#     duration = duration_tag[0].text

#     cut_tag = div_tag.find_elements(By.CSS_SELECTOR,'span.cut.svelte-17mbxpf')
#     cut = cut_tag[1].text

#     price_tag = div_tag.find_elements(By.CSS_SELECTOR,'div.price.svelte-17mbxpf div')
#     reg_price = price_tag[0].text
#     discount_price = price_tag[2].text
#     price_comp = price_tag[3].text

#     log = {'할인 시작 날짜':discount_date, '플랫폼 이름': store,'할인 지속시간':duration,'할인율':cut,'원가':reg_price,'할인가':discount_price,'이전과 비교한 할인가격':price_comp}
#     pprint(log)
#     # processed_log=process_price_data_with_sign(log)

# #5초로 waiting time 설정
# time.sleep(5)
# #driver 종료
# driver.quit()



In [ ]:
# import pandas as pd
# import re

# def clean_game_name_final(name):
#     """
#     게임 이름 문자열을 최종 클리닝하는 함수:
#     - 모두 소문자로 변환
#     - '&'를 'and'로 변환
#     - 띄어쓰기를 하이픈으로 변환
#     - 영어 소문자, 숫자, 하이픈 외 모든 문자 제거
#     - 연속된 하이픈 합치기 및 불필요한 하이픈 제거
#     """
#     # 0. 입력값이 NaN일 경우 빈 문자열로 처리
#     if pd.isna(name):
#         return ""
        
#     # 문자열로 변환하고 모두 소문자로 변경
#     cleaned_name = str(name).lower()

#     # 1. '&' 기호를 'and'로 변환 (먼저 처리)
#     cleaned_name = cleaned_name.replace('&', 'and')

#     # 2. 띄어쓰기(' ')를 하이픈('-')으로 변환
#     # 이 단계를 모든 비-a-z0-9- 문자 제거 전에 수행하여 공백이 하이픈으로 변환되도록 합니다.
#     cleaned_name = cleaned_name.replace(' ', '-')

#     # 3. 영어 소문자, 숫자, 하이픈('-')을 제외한 모든 문자 제거
#     # 이제 공백은 하이픈으로 변환되었으므로, 이 정규식은 하이픈을 제외한 나머지 불필요한 문자만 제거합니다.
#     cleaned_name = re.sub(r'[^a-z0-9-]', '', cleaned_name)

#     # 4. 연속으로 나타나는 하이픈을 하나로 줄이기 (예: 'metal--gear' -> 'metal-gear')
#     # 이전 단계에서 공백이 하이픈으로 바뀌고, 다른 문자들이 제거되면서 연속 하이픈이 생길 수 있습니다.
#     cleaned_name = re.sub(r'-+', '-', cleaned_name)
    
#     # 5. 문장 시작/끝에 불필요하게 붙는 하이픈 제거 (예: '-metal-gear-' -> 'metal-gear')
#     cleaned_name = cleaned_name.strip('-')
    
#     # 6. 마지막으로, 혹시 남아있을 수 있는 공백 제거 (trim)
#     cleaned_name = cleaned_name.strip()

#     return cleaned_name
